## 🔗 Open This Notebook in Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DavidLangworthy/ds4s/blob/main/days/day04/notebook/day04_starter.ipynb)

# 🌍 Day 4 – Mapping Forest Change and Habitat Pressure

Today you will transform a long-format forest cover dataset into static and animated maps that illustrate biodiversity risk.

### Data Card — World Bank Forest Area (% of land)

| Field | Details |
| --- | --- |
| Source | World Bank World Development Indicators (`AG.LND.FRST.ZS`) |
| File | `data/forest_area_long.csv` |
| Temporal coverage | 1990–2020 |
| Spatial coverage | Countries (ISO-3) plus regional aggregates |
| Units | Percent of land area covered by forest |
| Last updated | November 2024 data release |
| Caveats | Regional aggregates remain; filter to ISO codes to avoid double counting. Values may be revised retroactively. |


### Step 1 · Imports and helpers

We will rely on Plotly for mapping and pandas for wrangling.

In [ ]:
import pandas as pd
import plotly.express as px

from utils import (
    expect_rows_between,
    load_csv,
    quick_check,
    save_plotly_fig,
    validate_columns,
    validate_story_elements,
)


### Step 2 · Load the forest dataset and inspect the shape

Confirm the expected columns before filtering.

In [ ]:
forest_raw = load_csv("data/forest_area_long.csv")
quick_check(forest_raw.head(), name="Forest preview")
validate_columns(forest_raw, ["Country Name", "Country Code", "Year", "ForestPercent"])


### Step 3 · Focus on country-level rows

Filter to ISO-style codes and ensure the numeric types are clean.

In [ ]:
forest_countries = forest_raw[forest_raw["Country Code"].str.fullmatch(r"[A-Z]{3}")].copy()
forest_countries["Year"] = forest_countries["Year"].astype(int)
forest_countries["ForestPercent"] = pd.to_numeric(forest_countries["ForestPercent"], errors="coerce")
forest_countries = forest_countries.dropna(subset=["ForestPercent"])
expect_rows_between(forest_countries, minimum=4000, maximum=6000)
quick_check(forest_countries.tail(), name="Country forest data")


### Step 4 · Build a data slice for a single year (static map)

Pick a recent year to help students anchor the animation.

In [ ]:
latest_year = 2020
forest_latest = forest_countries[forest_countries["Year"] == latest_year]
expect_rows_between(forest_latest, minimum=180, maximum=210)
quick_check(forest_latest.head(), name=f"Forest cover {latest_year}")


### Step 5 · Prepare story scaffolding

The same checklist reinforces intentional titles and annotations.

In [ ]:
story = {
    "title": "Forests Shrink in Key Biodiversity Hotspots",
    "subtitle": f"Share of land area covered by forest, {latest_year}",
    "annotation": "Amazon basin countries show <55% forest cover despite vast rainforests.",
    "source": "World Bank WDI AG.LND.FRST.ZS (retrieved 2024-11)",
    "units": "% of land area",
}
validate_story_elements(story)


### Step 6 · Static choropleth for the latest year

Expect tropical regions to appear darkest green; arid regions lighter.

In [ ]:
fig_static = px.choropleth(
    forest_latest,
    locations="Country Code",
    color="ForestPercent",
    hover_name="Country Name",
    color_continuous_scale="Greens",
    range_color=[0, 100],
    labels={"ForestPercent": story["units"]},
    title=f"{story['title']} — {story['subtitle']}",
)
fig_static.update_layout(margin=dict(l=0, r=0, t=60, b=0))
fig_static.show()


### Step 7 · Animated choropleth over time

The animation slider should reveal declines in South America and Southeast Asia.

In [ ]:
fig_animated = px.choropleth(
    forest_countries,
    locations="Country Code",
    color="ForestPercent",
    hover_name="Country Name",
    animation_frame="Year",
    color_continuous_scale="Greens",
    range_color=[0, 100],
    labels={"ForestPercent": story["units"]},
    title="Forest cover (% of land) by country, 1990–2020",
)
fig_animated.update_layout(margin=dict(l=0, r=0, t=60, b=0))
fig_animated.show()


### Step 8 · Export the animation HTML for archiving

The HTML retains interactivity without embedding heavy JSON in the notebook.

In [ ]:
save_plotly_fig(fig_animated, "plots/day04_solution_forest.html")


### Step 9 · Reflection prompts

- Where do you observe the steepest declines, and what biodiversity impacts follow?
- How could you annotate the animation to highlight tipping points?
- Note at least one limitation (e.g., tree plantations counted as forest).